## Data importation

In [20]:
import pandas as pd
import numpy as np

data_raw = pd.read_csv("data.csv", delimiter=";", encoding="utf-8")
data = np.array(data_raw)

weights = [4, 3, 5, 3, 5, 1, 12, 9]
data = data[: , 3:] # we remove program and university name columns

## First application of ELECTRE I

In [21]:
from pyDecision.algorithm import electre_i


c_hat = 0.65
d_hat = 0.35

concordance, discordance, dominance, kernel, dominated = electre_i(data, W = weights, c_hat = c_hat, d_hat = d_hat, graph=False)

## Robustness study

In [22]:

kernels = []

for c_hat in np.arange(0, 1.1, 0.1):

	for d_hat in np.arange(0, 1.1, 0.1):
		try:
			concordance, discordance, dominance, kernel, dominated = electre_i(data, W = weights, c_hat = c_hat, d_hat = d_hat, graph = False)
			kernels.extend(kernel)
		except IndexError:
			pass

In [23]:
ker = np.array(kernels)
actions = [f"a{val}" for val in range(1, 44)]
programs = [f"{np.array(data_raw)[:, 1][elt]} - {np.array(data_raw)[:, 2][elt]}" for elt in range(len(data_raw))]
results = [{"program": program, "occurrences": np.count_nonzero(ker == action)} for action, program in zip(actions, programs)]

results = sorted(results, key=lambda x: x["occurrences"], reverse=True)

In [24]:
import plotly.express as px
fig = px.bar(results, x="program", y="occurrences")
fig.update_xaxes(showticklabels=False) # we hide x labels because it takes too much room. We can see school names by hovering bars
fig.show()